In [1]:
import torch
import torch.nn as nn

from torchvision.datasets import MNIST
from torchvision import transforms

from torchsummary import summary

import matplotlib.pyplot as plt

In [2]:
train = MNIST('data', train=True, transform=transforms.ToTensor(), download=True)
test = MNIST('data', train=False, transform=transforms.ToTensor())

  0%|          | 16384/9912422 [00:00<01:16, 129608.28it/s]

9920512it [00:00, 37251047.14it/s]                           


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


32768it [00:00, 970429.04it/s]
1654784it [00:00, 11462457.70it/s]                         

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



8192it [00:00, 367468.11it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [3]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True) 
test_loader = torch.utils.data.DataLoader(test, batch_size=128) 

In [4]:
n_input = 784
n_dense = 64
n_out = 10

In [5]:
model = nn.Sequential(
    nn.Linear(n_input, n_dense), # hidden layer
    nn.Sigmoid(), # activation function
    nn.Linear(n_dense, n_out) # output layer
)

In [6]:
summary(model, (1, n_input))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 64]          50,240
           Sigmoid-2                [-1, 1, 64]               0
            Linear-3                [-1, 1, 10]             650
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.19
Estimated Total Size (MB): 0.20
----------------------------------------------------------------


In [7]:
cost_fxn = nn.CrossEntropyLoss() 

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [9]:
def accuracy_pct(pred_y, true_y):
  _, prediction = torch.max(pred_y, 1) # returns maximum values, indices; fed tensor, dim to reduce
  correct = (prediction == true_y).sum().item()
  return (correct / true_y.shape[0]) * 100.0

In [10]:
n_batches = len(train_loader)
n_batches

469

In [13]:
! python -V

Python 3.7.3


In [11]:
n_epochs = 20 

print('Training for {} epochs. \n'.format(n_epochs))

for epoch in range(n_epochs):
  
  avg_cost = 0.0
  avg_accuracy = 0.0
  
  for i, (X, y) in enumerate(train_loader): # enumerate() provides count of iterations  
    
    # forward propagation:
    X_flat = X.view(X.shape[0], -1)
    y_hat = model(X_flat)
    cost = cost_fxn(y_hat, y)
    avg_cost += cost / n_batches
    
    # backprop and optimization via gradient descent: 
    optimizer.zero_grad() # set gradients to zero; .backward() accumulates them in buffers
    cost.backward()
    optimizer.step()
    
    # calculate accuracy metric:
    accuracy = accuracy_pct(y_hat, y)
    avg_accuracy += accuracy / n_batches
    
    if (i + 1) % 100 == 0:
      print('Step {}'.format(i + 1))
    
  print('Epoch {}/{} complete: Cost: {:.3f}, Accuracy: {:.1f}% \n'
        .format(epoch + 1, n_epochs, avg_cost, avg_accuracy)) 
  # TO DO: add test metrics

print('Training complete.')

Training for 20 epochs. 

Step 100
Step 200
Step 300
Step 400
Epoch 1/20 complete: Cost: 2.238, Accuracy: 33.3% 

Step 100
Step 200
Step 300
Step 400
Epoch 2/20 complete: Cost: 2.064, Accuracy: 58.5% 

Step 100
Step 200
Step 300
Step 400
Epoch 3/20 complete: Cost: 1.804, Accuracy: 65.4% 

Step 100
Step 200
Step 300
Step 400
Epoch 4/20 complete: Cost: 1.509, Accuracy: 70.2% 

Step 100
Step 200
Step 300
Step 400
Epoch 5/20 complete: Cost: 1.258, Accuracy: 74.8% 

Step 100
Step 200


KeyboardInterrupt: 